In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy.exceptions import AxisError
import pandas as pd
from scipy import stats

from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Integer, String, insert, select

engine = create_engine("sqlite+pysqlite:///GPSdata1.db", echo=False)

metadata_obj = MetaData()

players_table = Table("players", metadata_obj, autoload_with=engine)
sessions_data_table = Table("sessions_data", metadata_obj, autoload_with=engine)

In [4]:
measure = "HSR/min"
measures = sessions_data_table.columns
print(measures.keys())

['player_id', 'match_id', 'Duration', 'Distance', 'HSR', 'SD', 'Accels + Decels', 'PlayerLoad', 'Top Speed', 'Dis/min', 'HSR/min', 'SD/min', 'Accels + Decels /min']


In [6]:
measure = "HSR/min"
measures = sessions_data_table.columns
HSR_columns = measures[measure]

with engine.connect() as conn:
    stmt = (
        select(players_table.c.team, players_table.c.position, measures)
        .join(sessions_data_table, players_table.c.id == sessions_data_table.c.player_id)
        .order_by(players_table.c.team)
    )

    data = conn.execute(stmt).mappings().all()

df = pd.DataFrame(data)
df

,Accels + Decels,Accels + Decels /min,Dis/min,Distance,Duration,HSR,HSR/min,PlayerLoad,SD,SD/min,Top Speed,match_id,player_id,position,team
0,141,1.473611,99.460900,9516.8,95.683333,282.8,2.955583,352.6516,124.0,1.295941,28.36820,1,1,CB,1st
1,120,1.233510,98.471900,9579.7,97.283333,465.3,4.782936,345.9358,109.3,1.123522,30.27220,2,1,CB,1st
2,140,1.481743,97.032400,9167.9,94.483333,249.9,2.644911,348.6969,80.5,0.852002,27.88820,3,1,CB,1st
3,132,1.328191,98.878100,9826.8,99.383333,385.1,3.874895,365.7599,75.5,0.759685,28.60820,4,1,CB,1st
4,123,1.245570,98.803200,9756.8,98.750000,440.9,4.464810,360.1288,47.0,0.475949,30.12820,5,1,CB,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,257,2.492726,96.219205,9920.2,103.100000,726.5,7.046557,388.7173,198.0,1.920466,31.00824,50,38,FWD,U23
559,208,2.234157,101.198711,9421.6,93.100000,696.8,7.484425,358.3715,325.4,3.495166,31.40028,32,38,FWD,U23
560,176,1.726054,108.048055,11017.3,101.966667,811.0,7.953580,404.1736,284.8,2.793070,30.76812,62,38,FWD,U23
561,232,2.412897,99.970879,9612.2,96.150000,679.2,7.063963,376.9603,113.3,1.178367,28.76832,55,38,FWD,U23


In [8]:
table_mean = table = pd.pivot_table(df, values=['HSR/min', 'Dis/min', 'SD/min', 'Accels + Decels /min', 'PlayerLoad'],
                       columns=['team'], aggfunc="mean").round(decimals=2)
table_mean

team,1st,U17,U18,U23
Accels + Decels /min,1.99,2.07,1.93,1.98
Dis/min,108.22,110.02,111.35,105.82
HSR/min,6.19,5.90,5.40,5.88
PlayerLoad,454.40,441.73,458.49,432.95
SD/min,2.01,1.06,1.13,1.52


In [47]:
table_mean = pd.pivot_table(df, values=['HSR/min', 'Dis/min', 'SD/min', 'Accels + Decels /min', 'PlayerLoad'],
                       columns=['team', 'position'], margins=False, margins_name="Team total", aggfunc="mean").round(decimals=2)

table_mean["Overall total"] = table_mean.mean(axis="columns").round(decimals=2)
table_mean

team                     1st                             U17                  \
position                  CB      CM      FB     FWD      CB      CM      FB   
Accels + Decels /min    1.42    2.38    2.21    1.95    1.62    2.19    2.69   
Dis/min                96.28  122.49  110.38  104.74  100.13  114.67  114.84   
HSR/min                 3.54    7.76    6.79    6.68    3.18    5.58    7.21   
PlayerLoad            384.01  519.87  484.05  433.74  392.72  465.24  481.00   
SD/min                  0.90    2.08    2.24    2.75    0.31    0.46    1.21   

team                             U18                             U23          \
position                 FWD      CB      CM      FB     FWD      CB      CM   
Accels + Decels /min    1.94    1.89    1.77    2.13    2.25    1.51    2.04   
Dis/min               107.99  105.86  116.22  115.06  115.21  102.83  107.82   
HSR/min                 7.61    4.88    4.78    7.10    7.16    4.06    5.43   
PlayerLoad            424.96  428.29  480.83  522.00  462.69  422.03  456.45   
SD/min                  2.32    0.93    0.65    1.98    2.20    0.77    0.97   

team                                 Overall total  
position                  FB     FWD                
Accels + Decels /min    2.13    2.12          2.01  
Dis/min               110.07  101.97        109.16  
HSR/min                 7.29    6.94          6.00  
PlayerLoad            458.48  387.07        450.21  
SD/min                  2.41    2.30          1.53

In [49]:
table_std_margins = pd.pivot_table(df, values=['HSR/min', 'Dis/min', 'SD/min', 'Accels + Decels /min', 'PlayerLoad'],
                       columns=['team', 'position'], margins=True, margins_name="Team total", aggfunc="std").round(decimals=2)

table_std_no_margins = pd.pivot_table(df, values=['HSR/min', 'Dis/min', 'SD/min', 'Accels + Decels /min', 'PlayerLoad'],
                       columns=['team', 'position'], margins=False, aggfunc="std").round(decimals=2)

table_std_margins["Overall total"] = table_std_no_margins.std(axis="columns").round(decimals=2)
table_std_margins

team                    1st                                    U17         \
position                 CB     CM     FB    FWD Team total     CB     CM   
Accels + Decels /min   0.16   0.36   0.27   0.57       0.17   0.30   0.47   
Dis/min                2.82   7.00   6.57   8.09       2.29   4.68   8.29   
HSR/min                0.71   1.98   1.65   2.03       0.61   0.98   2.45   
PlayerLoad            20.67  36.15  20.14  35.06       8.79  25.32  45.88   
SD/min                 0.36   1.32   0.78   1.23       0.44   0.27   0.31   

team                                           ...    U18                \
position                 FB    FWD Team total  ...     CM     FB    FWD   
Accels + Decels /min   0.32   0.42       0.08  ...   0.49   0.36   0.43   
Dis/min                3.98   8.00       2.23  ...   6.04   5.87  12.92   
HSR/min                0.92   1.86       0.74  ...   1.89   1.60   1.54   
PlayerLoad            15.61  50.58      16.62  ...  29.19  24.59  33.85   
SD/min                 0.23   1.53       0.63  ...   0.47   0.63   0.79   

team                               U23                                  \
position             Team total     CB     CM     FB    FWD Team total   
Accels + Decels /min       0.08   0.25   0.37   0.39   0.51       0.11   
Dis/min                    3.41   5.68   7.58   7.07   7.25       0.84   
HSR/min                    0.27   0.76   1.59   1.64   1.38       0.40   
PlayerLoad                 4.20  29.50  39.61  43.55  33.39       6.27   
SD/min                     0.14   0.43   0.70   1.25   1.01       0.36   

team                 Overall total  
position                            
Accels + Decels /min          0.11  
Dis/min                       2.23  
HSR/min                       0.49  
PlayerLoad                    9.84  
SD/min                        0.41  

[5 rows x 21 columns]

In [11]:
table_mean_plusequals = table_mean.astype(str) + " ± " + table_std.astype(str)
table_mean_plusequals

team                             1st                                  \
position                          CB              CM              FB   
Accels + Decels /min     1.42 ± 0.16     2.38 ± 0.36     2.21 ± 0.27   
Dis/min                 96.28 ± 2.82    122.49 ± 7.0   110.38 ± 6.57   
HSR/min                  3.54 ± 0.71     7.76 ± 1.98     6.79 ± 1.65   
PlayerLoad            384.01 ± 20.67  519.87 ± 36.15  484.05 ± 20.14   
SD/min                    0.9 ± 0.36     2.08 ± 1.32     2.24 ± 0.78   

team                                                            U17  \
position                         FWD     Team total              CB   
Accels + Decels /min     1.95 ± 0.57    1.99 ± 0.17      1.62 ± 0.3   
Dis/min                104.74 ± 8.09  108.47 ± 2.29   100.13 ± 4.68   
HSR/min                  6.68 ± 2.03    6.19 ± 0.61     3.18 ± 0.98   
PlayerLoad            433.74 ± 35.06  455.42 ± 8.79  392.72 ± 25.32   
SD/min                   2.75 ± 1.23    1.99 ± 0.44     0.31 ± 0.27   

team                                                                 \
position                          CM             FB             FWD   
Accels + Decels /min     2.19 ± 0.47    2.69 ± 0.32     1.94 ± 0.42   
Dis/min                114.67 ± 8.29  114.84 ± 3.98    107.99 ± 8.0   
HSR/min                  5.58 ± 2.45    7.21 ± 0.92     7.61 ± 1.86   
PlayerLoad            465.24 ± 45.88  481.0 ± 15.61  424.96 ± 50.58   
SD/min                   0.46 ± 0.31    1.21 ± 0.23     2.32 ± 1.53   

team                                  ...             U18                 \
position                  Team total  ...              CM             FB   
Accels + Decels /min     2.11 ± 0.08  ...     1.77 ± 0.49    2.13 ± 0.36   
Dis/min                109.41 ± 2.23  ...   116.22 ± 6.04  115.06 ± 5.87   
HSR/min                  5.89 ± 0.74  ...     4.78 ± 1.89      7.1 ± 1.6   
PlayerLoad            440.98 ± 16.62  ...  480.83 ± 29.19  522.0 ± 24.59   
SD/min                   1.08 ± 0.63  ...     0.65 ± 0.47    1.98 ± 0.63   

team                                                           U23  \
position                         FWD     Team total             CB   
Accels + Decels /min     2.25 ± 0.43    2.01 ± 0.08    1.51 ± 0.25   
Dis/min               115.21 ± 12.92  113.09 ± 3.41  102.83 ± 5.68   
HSR/min                  7.16 ± 1.54    5.98 ± 0.27    4.06 ± 0.76   
PlayerLoad            462.69 ± 33.85   473.45 ± 4.2  422.03 ± 29.5   
SD/min                    2.2 ± 0.79    1.44 ± 0.14    0.77 ± 0.43   

team                                                                  \
position                          CM              FB             FWD   
Accels + Decels /min     2.04 ± 0.37     2.13 ± 0.39     2.12 ± 0.51   
Dis/min                107.82 ± 7.58   110.07 ± 7.07   101.97 ± 7.25   
HSR/min                  5.43 ± 1.59     7.29 ± 1.64     6.94 ± 1.38   
PlayerLoad            456.45 ± 39.61  458.48 ± 43.55  387.07 ± 33.39   
SD/min                    0.97 ± 0.7     2.41 ± 1.25      2.3 ± 1.01   

team                                  Overall total  
position                 Team total                  
Accels + Decels /min    1.95 ± 0.11     2.01 ± 0.14  
Dis/min               105.67 ± 0.84   109.16 ± 2.76  
HSR/min                  5.93 ± 0.4      6.0 ± 0.61  
PlayerLoad            431.01 ± 6.27  450.21 ± 12.99  
SD/min                  1.61 ± 0.36      1.53 ± 0.4  

[5 rows x 21 columns]

In [53]:
HSR_mean = pd.pivot_table(df, values=['HSR/min'],
                       columns=['team', 'position'], margins=True, margins_name="Team total", aggfunc="mean").round(decimals=2)
HSR_mean["Overall total"] = HSR_mean.mean(axis="columns").round(decimals=2)



HSR_mean

team       1st                                U17                    \
position    CB    CM    FB   FWD Team total    CB    CM    FB   FWD   
HSR/min   3.54  7.76  6.79  6.68       6.19  3.18  5.58  7.21  7.61   

team                 ...   U18                         U23                    \
position Team total  ...    CM   FB   FWD Team total    CB    CM    FB   FWD   
HSR/min        5.89  ...  4.78  7.1  7.16       5.98  4.06  5.43  7.29  6.94   

team                Overall total  
position Team total                
HSR/min        5.93           6.0  

[1 rows x 21 columns]

In [54]:
HSR_std = pd.pivot_table(df, values=['HSR/min'],
                       columns=['team', 'position'], margins=True, margins_name="Team total", aggfunc="std").round(decimals=2)


HSR_std["Overall total"] = HSR_std.std(axis="columns").round(decimals=2)

HSR_mean_std = pd.concat([HSR_mean, HSR_std])
HSR_mean_std.insert(0,"", ["Mean", "SD"], allow_duplicates=True)
HSR_mean_std

team             1st                                U17                    \
position          CB    CM    FB   FWD Team total    CB    CM    FB   FWD   
HSR/min   Mean  3.54  7.76  6.79  6.68       6.19  3.18  5.58  7.21  7.61   
HSR/min     SD  0.71  1.98  1.65  2.03       0.61  0.98  2.45  0.92  1.86   

team      ...   U18                         U23                               \
position  ...    CM   FB   FWD Team total    CB    CM    FB   FWD Team total   
HSR/min   ...  4.78  7.1  7.16       5.98  4.06  5.43  7.29  6.94       5.93   
HSR/min   ...  1.89  1.6  1.54       0.27  0.76  1.59  1.64  1.38       0.40   

team     Overall total  
position                
HSR/min           6.00  
HSR/min           0.61  

[2 rows x 22 columns]

In [50]:
def generate_mean_sd_table(measure: str, dataframe: pd.DataFrame):

    mean_table = pd.pivot_table(dataframe, values=[measure],
                       columns=['team', 'position'], margins=True, margins_name="Team total", aggfunc="mean").round(decimals=2)
    mean_table["Overall total"] = mean_table.mean(axis="columns").round(decimals=2)

    std_table = pd.pivot_table(dataframe, values=[measure],
                       columns=['team', 'position'], margins=True, margins_name="Team total", aggfunc="std").round(decimals=2)
    
    # No margins are required to team grouped stds do not intefere with whole sample std
    std_table_no_margines = pd.pivot_table(dataframe, values=[measure],
                       columns=['team', 'position'], margins=True, margins_name="Team total", aggfunc="std").round(decimals=2)
    
    std_table["Overall total"] = std_table_no_margines.std(axis="columns").round(decimals=2)


    new_table = pd.concat([mean_table, std_table])
    new_table.insert(0,"", ["Mean", "SD"], allow_duplicates=True)
    return new_table

In [56]:
measures = ['HSR/min', 'Dis/min', 'SD/min', 'Accels + Decels /min', 'PlayerLoad']

sub_tables = []

for measure in measures:
    sub_tables.append(generate_mean_sd_table(measure, df))

        
table = pd.concat(sub_tables)
table["1st"]

position,CB,CM,FB,FWD,Team total
HSR/min,3.54,7.76,6.79,6.68,6.19
HSR/min,0.71,1.98,1.65,2.03,0.61
Dis/min,96.28,122.49,110.38,104.74,108.47
Dis/min,2.82,7.00,6.57,8.09,2.29
SD/min,0.90,2.08,2.24,2.75,1.99
SD/min,0.36,1.32,0.78,1.23,0.44
Accels + Decels /min,1.42,2.38,2.21,1.95,1.99
Accels + Decels /min,0.16,0.36,0.27,0.57,0.17
PlayerLoad,384.01,519.87,484.05,433.74,455.42
PlayerLoad,20.67,36.15,20.14,35.06,8.79


In [16]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [126]:
wb = Workbook()
ws = wb.active
ws.title = "Data table"

rows = dataframe_to_rows(table)

for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)

wb.save("test.xlsx")
